# How to solve a linear system using VQLS amd precomputed quantum circuits

This how-to guide explain how to sovle a linear systems of equations using VQLS when you already have quantum circuits to represent your matrix $A$ and your vector $b$

## Step 1: Create the quantum circuits of your linear system

Let's assume that your linear system is represented by the matrix $A$ defined by:

$$
A = \frac{1}{2}(X_0 X_1 \text{CNOT}(0,1) + H_0X_1 \text{CNOT}(0,1))
$$

and that the be vector is given by:

$$
|b\rangle = H_0H_1|0\rangle
$$

It is of course possible to use these circuits to solve the linear system. To this end we simply need to use them as input of the `.solve()` method. Let's create these circuits

In [1]:
from qiskit import QuantumCircuit

# number of qbit in the circuit
nqbit = 2

# first quantum circuit for A
qc1 = QuantumCircuit(nqbit)
qc1.x(0)
qc1.x(1)
qc1.cx(0, 1)

# second quantum circuit for A
qc2 = QuantumCircuit(nqbit)
qc2.h(0)
qc2.x(1)
qc2.cx(0, 1)

# quantum circuit for b
rhs = QuantumCircuit(nqbit)
rhs.h(0)
rhs.h(1)

## Step 2 : Create the solver
As for the previous tutorial we weill firs create the solver. 

In [2]:
from vqls_prototype import VQLS, VQLSLog
from qiskit.primitives import Estimator
from qiskit_algorithms import optimizers as opt
from qiskit.circuit.library.n_local.real_amplitudes import RealAmplitudes


ansatz = RealAmplitudes(nqbit, entanglement="full", reps=3, insert_barriers=False)

# instantiate an estimator primitive
estimator = Estimator()

# create a logger
log = VQLSLog([], [])

# create the vqls solver
vqls = VQLS(estimator, ansatz, opt.CG(maxiter=200))

We can now call the `.solve()` method and use a `List` of `(float, QuantumCircuit)` for the matrix and the `QuantumCircuit`  for the vector.

In [3]:
solution = vqls.solve([[0.5, qc1], [0.5, qc2]], rhs)